In [1]:
import os
import re
import legacy
import click
import dnnlib
import numpy as np
import PIL.Image
import torch
import matplotlib.pyplot as plt
import random

from sklearn.decomposition import PCA
from torch_utils import misc
from torch_utils import persistence
from torch_utils.ops import conv2d_resample
from torch_utils.ops import upfirdn2d
from torch_utils.ops import bias_act
from torch_utils.ops import fma
from typing import List, Optional

In [2]:
url = 'D:\Rutgers_Study_Files\\2021Fall\\535\\final_project\\network-snapshot-010483.pkl'
device = torch.device('cuda')
outdir = 'D:\\Rutgers_Study_Files\\2021Fall\\535\\final_project\\generated'
outdir_2 = 'D:\\Rutgers_Study_Files\\2021Fall\\535\\final_project\\generated_2'
device = torch.device('cuda')
with dnnlib.util.open_url(url) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device)  # type: ignore
os.makedirs(outdir, exist_ok=True)

In [3]:
def expand_seed(seeds):
    result = []
    vector_size = G.z_dim
    for seed in seeds:
        rnd = np.random.RandomState(seed)
        result.append(rnd.randn(1, vector_size))
    return np.array(result)

In [4]:
def generate_seeds(size):
    seeds = []
    for i in range(size):
        seeds.append(random.randint(0,100000))
    expanded_seeds = expand_seed(seeds)
    return expanded_seeds

In [5]:
def pca_basis(G, size = 10000):
    """
    G: G model
    size: sample size (bigger, better)
    """
    c = None
    z = generate_seeds(size)
#     z = torch.tensor(z)
#     print(z.shape)
#     z = torch.randn([1, G.z_dim]).cuda()    
    w = []
    for i in range(size):
        w.append(G.mapping(torch.tensor(z[i]).cuda(), c, truncation_psi=0.5, truncation_cutoff=8).cpu().numpy().tolist()) # not sure output size
    # if w is 10000*p, then use the following code; otherwise, transfer to 10000*p first
    w = np.array(w)
    w = np.reshape(w,(size, 14 * 512))
    pca = PCA()
    pca.fit(w)
    ratio = pca.explained_variance_ratio_
    v = pca.components_
    return v, ratio

v, ratio = pca_basis(G)

Setting up PyTorch plugin "bias_act_plugin"... Failed!


C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\bias_act.py:50: UserWarning: Failed to build CUDA kernels for bias_act. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\bias_act.py", line 48, in _init
    _plugin = custom_ops.get_plugin('bias_act_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py", line 64, in get_plugin
    raise RuntimeError(f'Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "{__file__}".')
RuntimeError: Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py".

  warnings.warn('Failed to build CUDA kernels for bias_act. Falling back to slow reference implementation. Details:\n\n' + traceback.form

In [6]:
ratio

array([9.30027629e-02, 6.98511992e-02, 6.96037859e-02, ...,
       1.64253376e-36, 6.52220093e-37, 1.40172936e-37])

In [25]:
rr = ratio.tolist()

In [28]:
with open('D:\\Rutgers_Study_Files\\2021Fall\\535\\final_project\\ratio.txt', 'wt') as f:
    for i in rr:
        print(i, file=f)
f.close()

In [24]:
def steer_with_direction(G,startpoint,v,nth,num_step,size_step):#return seires of result output and show
    """
    G: G model
    startpoint: input seed (shape [1, seed.size])
    v: pca basis of latent space
    nth: number-th of pc loading used to update latent space
    num_step: number of image generated (control parameter value)
    size_step: step change of pc basis (control parameter value) (list)
    """
    c = None
    startpoint = startpoint.to(device)
    direction = torch.from_numpy(v[nth]).to(device)
    f, axarr = plt.subplots(1,num_step, figsize=(4*num_step,4), sharex='all', sharey='all')
    for i in range(num_step):
        w = G.mapping(startpoint, c, truncation_psi=0.5, truncation_cutoff=8).cpu()
        w = np.reshape(w,(1, 14 * 512))
        input = w.cuda() + size_step[i] * direction
        input = np.reshape(input.cpu(), (1, 14, 512))
        input = input.float().to(device)
        img = G.synthesis(input, noise_mode='const', force_fp32=True)
        img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{seeds[seed_idx]:04d}.png')
        axarr[i].imshow(img[0].cpu(), cmap='Blues',interpolation='nearest')
        axarr[i].axis('off')
    f.tight_layout(pad = 0.5)
    plt.show()

In [10]:
def steer_with_direction_generate(G,startpoint,v):#return seires of result output and show
    """
    G: G model
    startpoint: input seed (shape [1, seed.size])
    v: pca basis of latent space
    nth: number-th of pc loading used to update latent space
    num_step: number of image generated (control parameter value)
    size_step: step change of pc basis (control parameter value) (list)
    """
    c = None
    i = -150
    for j in range(len(startpoint)):
        nth = random.randint(0,200)
        if i == -150:
            i = 150
        else:
            i = -150
        sp = torch.Tensor(startpoint[j])
        sp = sp.to(device)
        direction = torch.from_numpy(v[nth]).to(device)
        w = G.mapping(sp, c, truncation_psi=0.5, truncation_cutoff=8).cpu()
        w = np.reshape(w,(1, 14 * 512))
        input = w.cuda() + i * direction
        input = np.reshape(input.cpu(), (1, 14, 512))
        input = input.float().to(device)
        img = G.synthesis(input, noise_mode='const', force_fp32=True)
        img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{j:04d}.png')

In [11]:
def test():
    z = generate_seeds(2500)
#     nth = 2000
#     num_step = 1
#     size_step = np.linspace(-10, 10, 1)
#     steer_with_direction(G,z,v,nth,num_step,size_step)
    steer_with_direction_generate(G,z,v)

In [ ]:
test()